# 循环神经网络的从零开始实现
:label:`sec_rnn_scratch`

本节将根据 :numref:`sec_rnn`中的描述，
从头开始基于循环神经网络实现字符级语言模型。
这样的模型将在H.G.Wells的时光机器数据集上训练。
和前面 :numref:`sec_language_model`中介绍过的一样，
我们先读取数据集。


In [ ]:
%matplotlib inline
import math
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

In [ ]:
batch_size, num_steps = 2, 35
train_iter, vocab = d2l.load_data_time_machine(batch_size, num_steps)

In [ ]:
print(len(vocab))

## [**独热编码**]

回想一下，在`train_iter`中，每个词元都表示为一个数字索引，
将这些索引直接输入神经网络可能会使学习变得困难。
我们通常将每个词元表示为更具表现力的特征向量。
最简单的表示称为*独热编码*（one-hot encoding），
它在 :numref:`subsec_classification-problem`中介绍过。

简言之，将每个索引映射为相互不同的单位向量：
假设词表中不同词元的数目为$N$（即`len(vocab)`），
词元索引的范围为$0$到$N-1$。
如果词元的索引是整数$i$，
那么我们将创建一个长度为$N$的全$0$向量，
并将第$i$处的元素设置为$1$。
此向量是原始词元的一个独热向量。
索引为$0$和$2$的独热向量如下所示：


In [ ]:
t = torch.tensor([0,2])
print(t)
F.one_hot(t, len(vocab))

我们每次采样的(**小批量数据形状是二维张量：
（批量大小，时间步数）。**)
`one_hot`函数将这样一个小批量数据转换成三维张量，
张量的最后一个维度等于词表大小（`len(vocab)`）。
我们经常转换输入的维度，以便获得形状为
（时间步数，批量大小，词表大小）的输出。
这将使我们能够更方便地通过最外层的维度，
一步一步地更新小批量数据的隐状态。


In [ ]:
X = torch.arange(10).reshape((2, 5))
print(X)
F.one_hot(X.T, 28).shape
print(F.one_hot(X.T, 28))

## 初始化模型参数

接下来，我们[**初始化循环神经网络模型的模型参数**]。
隐藏单元数`num_hiddens`是一个可调的超参数。
当训练语言模型时，输入和输出来自相同的词表。
因此，它们具有相同的维度，即词表的大小。


In [ ]:
def get_params(vocab_size, num_hiddens, device):
    """
    初始化循环神经网络的参数。

    参数:
        vocab_size: 词表大小，同时作为输入和输出的特征数
        num_hiddens: 隐藏单元数
        device: 参数所在的设备（如 'cpu' 或 'cuda'）

    返回:
        包含所有参数的列表，每个参数都设置了 requires_grad=True
    """
    num_inputs = num_outputs = vocab_size  # 输入和输出的特征数都等于词表大小

    def normal(shape):
        # 生成服从正态分布的张量，均值为0，标准差为0.01
        return torch.randn(size=shape, device=device) * 0.01

    # 隐藏层参数
    W_xh = normal((num_inputs, num_hiddens))  # 输入到隐藏层的权重
    W_hh = normal((num_hiddens, num_hiddens)) # 隐藏层到隐藏层的权重
    b_h = torch.zeros(num_hiddens, device=device)  # 隐藏层偏置

    # 输出层参数
    W_hq = normal((num_hiddens, num_outputs))  # 隐藏层到输出层的权重
    b_q = torch.zeros(num_outputs, device=device)  # 输出层偏置

    # 将所有参数放入列表
    params = [W_xh, W_hh, b_h, W_hq, b_q]
    for param in params:
        # 设置参数需要计算梯度
        param.requires_grad_(True)
    return params

## 循环神经网络模型

为了定义循环神经网络模型，
我们首先需要[**一个`init_rnn_state`函数在初始化时返回隐状态**]。
这个函数的返回是一个张量，张量全用0填充，
形状为（批量大小，隐藏单元数）。
在后面的章节中我们将会遇到隐状态包含多个变量的情况，
而使用元组可以更容易地处理些。


In [ ]:
# 该函数用于初始化循环神经网络（RNN）的隐藏状态（hidden state）。
# 输入参数：
#   batch_size: 批量大小，即一次输入多少个样本
#   num_hiddens: 隐藏单元的数量
#   device: 张量所在的设备（如 'cpu' 或 'cuda'）
# 返回值：
#   返回一个元组，里面是一个形状为 (batch_size, num_hiddens) 的全零张量，表示初始的隐藏状态
def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), )

[**下面的`rnn`函数定义了如何在一个时间步内计算隐状态和输出。**]
循环神经网络模型通过`inputs`最外层的维度实现循环，
以便逐时间步更新小批量数据的隐状态`H`。
此外，这里使用$\tanh$函数作为激活函数。
如 :numref:`sec_mlp`所述，
当元素在实数上满足均匀分布时，$\tanh$函数的平均值为0。


In [ ]:
# 这段代码实现了一个简单的循环神经网络（RNN）的前向计算过程。
def rnn(inputs, state, params):
    # inputs: 输入序列，形状为 (时间步数, 批量大小, 词表大小)
    # state: 隐藏状态，通常是一个元组，这里只包含一个张量 H
    # params: 网络参数，包括输入到隐藏层、隐藏到隐藏层、隐藏层偏置、隐藏到输出层、输出层偏置

    W_xh, W_hh, b_h, W_hq, b_q = params  # 解包参数
    H, = state  # 当前的隐藏状态

    outputs = []  # 用于保存每个时间步的输出

    # 手动实现 RNN 内部计算
    # inputs(T,B,V)
    for X in inputs:
        # 计算当前时间步的隐藏状态
        # X(B,V)
        # H(B,H)
        # 隐藏层
        H = torch.tanh(torch.mm(X, W_xh) + torch.mm(H, W_hh) + b_h)

        # 计算当前时间步的输出
        # Y(B,V)
        # 输出层
        Y = torch.mm(H, W_hq) + b_q
        outputs.append(Y)  # 保存输出

    # 输出形状为 (T * B, V)
    O = torch.cat(outputs, dim=0)
    # 同时返回最后一个时间步的隐藏状态 H，将其封装为元组以保持与输入 state 相同的格式。
    return O, (H,)

定义了所有需要的函数之后，接下来我们[**创建一个类来包装这些函数**]，
并存储从零开始实现的循环神经网络模型的参数。


In [ ]:
class RNNModelScratch: #@save
    """从零开始实现的循环神经网络模型"""
    def __init__(self, vocab_size, num_hiddens, device, get_params, init_state, forward_fn):
        # 保存词表大小和隐藏单元数
        self.vocab_size, self.num_hiddens = vocab_size, num_hiddens
        # 获取模型参数
        self.params = get_params(vocab_size, num_hiddens, device)
        # 保存初始化隐状态和前向传播函数
        self.init_state, self.forward_fn = init_state, forward_fn

    def __call__(self, X, state):
        # 将输入X转为one-hot编码，并转为float32类型
        # X: (batch_size, num_steps) 独热编码 (T, B, V)
        X = F.one_hot(X.T, self.vocab_size).type(torch.float32)
        # 前向传播，返回输出和新的隐状态
        # 输出形状: (T * B, V)
        # 新的隐状态形状取决于具体实现，通常为 (B, H)
        return self.forward_fn(X, state, self.params)

    def begin_state(self, batch_size, device):
        # 初始化隐状态
        return self.init_state(batch_size, self.num_hiddens, device)

让我们[**检查输出是否具有正确的形状**]。
例如，隐状态的维数是否保持不变。


In [ ]:
# 定义RNN模型的隐藏层大小
num_hiddens = 512

# 创建RNN模型实例，使用从零开始实现的RNNModelScratch类
# 参数说明：
#   len(vocab) - 词汇表大小，即输入和输出的维度
#   num_hiddens - 隐藏层单元数量
#   d2l.try_gpu() - 尝试使用GPU设备进行计算
#   get_params - 获取模型参数的函数
#   init_rnn_state - 初始化隐藏状态的函数
#   rnn - RNN前向传播函数
net = RNNModelScratch(len(vocab), num_hiddens, d2l.try_gpu(), get_params, init_rnn_state, rnn)

# 初始化RNN的隐藏状态
# 根据输入批量大小和设备信息创建初始隐藏状态
state = net.begin_state(X.shape[0], d2l.try_gpu())

# 执行前向传播计算
# 将输入数据移到GPU上，并与初始隐藏状态一起传入模型进行计算
# 返回输出结果Y和更新后的隐藏状态new_state
Y, new_state = net(X.to(d2l.try_gpu()), state)


# 检查输出结果和隐藏状态的形状信息
print(Y.shape) # Y的shape是(num_steps * batch_size, vocab_size)
print(len(new_state)) 
print(new_state[0].shape) # new_state元素的 shape 是(batch_size,num_hiddens)

我们可以看到输出形状是（时间步数$\times$批量大小，词表大小），
而隐状态形状保持不变，即（批量大小，隐藏单元数）。

## 预测

让我们[**首先定义预测函数来生成`prefix`之后的新字符**]，
其中的`prefix`是一个用户提供的包含多个字符的字符串。
在循环遍历`prefix`中的开始字符时，
我们不断地将隐状态传递到下一个时间步，但是不生成任何输出。
这被称为*预热*（warm-up）期，
因为在此期间模型会自我更新（例如，更新隐状态），
但不会进行预测。
预热期结束后，隐状态的值通常比刚开始的初始值更适合预测，
从而预测字符并输出它们。


In [ ]:
# 预热->训练->预测
# prefix: 预热期的前缀字符串
# num_preds: 预热之后，预测的字符数
# net: 训练好的RNN模型
# vocab: 词汇表对象，用于将字符索引转换为字符
# device: 计算设备，如CPU或GPU
def predict_ch8(prefix, num_preds, net, vocab, device):  #@save
    # 初始化RNN的隐状态，batch_size=1表示一次生成一个序列
    state = net.begin_state(batch_size=1, device=device)
    # outputs用于保存生成的字符的索引，首先放入prefix的第一个字符的索引
    outputs = [vocab[prefix[0]]]
    # 定义一个函数，每次根据outputs的最后一个字符索引生成输入张量，shape(batch_size,num_steps)=(1,1)
    get_input = lambda: torch.tensor([outputs[-1]], device=device).reshape((1, 1))
    # 将输入张量reshape成(1, 1)是因为在PyTorch中，RNN模型通常期望输入具有特定的维度格式。
    # 这里第一个维度1表示batch_size为1（一次处理一个样本），第二个维度1表示序列长度为1（一次输入一个时间步的数据）。
    
    # 预热期：依次将prefix的后续字符输入到模型中，更新隐状态，但不预测新字符
    for y in prefix[1:]:  # 预热期
        _, state = net(get_input(), state)
        outputs.append(vocab[y])
    # 预测阶段：根据当前隐状态和上一个输出，预测接下来的num_preds个字符
    for _ in range(num_preds):  # 预测num_preds步
        y, state = net(get_input(), state)
        # y 的 shape 是 (t*b,vocab_size)=(1*1, vocab_size)，表示当前时间步的输出分布
        # print("当前预测的字符索引（在各可能类别中概率最大的索引）:", y.argmax(dim=1).reshape(1))
        # 取预测结果中概率最大的字符索引作为下一个输出
        outputs.append(int(y.argmax(dim=1).reshape(1)))
    # 将所有字符索引转换为字符并拼接成字符串返回
    return ''.join([vocab.idx_to_token[i] for i in outputs])

现在我们可以测试`predict_ch8`函数。
我们将前缀指定为`time traveller `，
并基于这个前缀生成10个后续字符。
鉴于我们还没有训练网络，它会生成荒谬的预测结果。


In [ ]:
predict_ch8('time traveller ', 20, net, vocab, d2l.try_gpu())

## [**梯度裁剪**]

对于长度为$T$的序列，我们在迭代中计算这$T$个时间步上的梯度，
将会在反向传播过程中产生长度为$\mathcal{O}(T)$的矩阵乘法链。
如 :numref:`sec_numerical_stability`所述，
当$T$较大时，它可能导致数值不稳定，
例如可能导致梯度爆炸或梯度消失。
因此，循环神经网络模型往往需要额外的方式来支持稳定训练。

一般来说，当解决优化问题时，我们对模型参数采用更新步骤。
假定在向量形式的$\mathbf{x}$中，
或者在小批量数据的负梯度$\mathbf{g}$方向上。
例如，使用$\eta > 0$作为学习率时，在一次迭代中，
我们将$\mathbf{x}$更新为$\mathbf{x} - \eta \mathbf{g}$。
如果我们进一步假设目标函数$f$表现良好，
即函数$f$在常数$L$下是*利普希茨连续的*（Lipschitz continuous）。
也就是说，对于任意$\mathbf{x}$和$\mathbf{y}$我们有：

$$|f(\mathbf{x}) - f(\mathbf{y})| \leq L \|\mathbf{x} - \mathbf{y}\|.$$

在这种情况下，我们可以安全地假设：
如果我们通过$\eta \mathbf{g}$更新参数向量，则

$$|f(\mathbf{x}) - f(\mathbf{x} - \eta\mathbf{g})| \leq L \eta\|\mathbf{g}\|,$$

这意味着我们不会观察到超过$L \eta \|\mathbf{g}\|$的变化。
这既是坏事也是好事。
坏的方面，它限制了取得进展的速度；
好的方面，它限制了事情变糟的程度，尤其当我们朝着错误的方向前进时。

有时梯度可能很大，从而优化算法可能无法收敛。
我们可以通过降低$\eta$的学习率来解决这个问题。
但是如果我们很少得到大的梯度呢？
在这种情况下，这种做法似乎毫无道理。
一个流行的替代方案是通过将梯度$\mathbf{g}$投影回给定半径
（例如$\theta$）的球来裁剪梯度$\mathbf{g}$。
如下式：

(**$$\mathbf{g} \leftarrow \min\left(1, \frac{\theta}{\|\mathbf{g}\|}\right) \mathbf{g}.$$**)

通过这样做，我们知道梯度范数永远不会超过$\theta$，
并且更新后的梯度完全与$\mathbf{g}$的原始方向对齐。
它还有一个值得拥有的副作用，
即限制任何给定的小批量数据（以及其中任何给定的样本）对参数向量的影响，
这赋予了模型一定程度的稳定性。
梯度裁剪提供了一个快速修复梯度爆炸的方法，
虽然它并不能完全解决问题，但它是众多有效的技术之一。

下面我们定义一个函数来裁剪模型的梯度，
模型是从零开始实现的模型或由高级API构建的模型。
我们在此计算了所有模型参数的梯度的范数。


In [ ]:
# 梯度裁切：这种裁剪方式称为 “按范数裁剪”（Clipping by Norm），是梯度裁剪中最常用的一种。
# 这相当于把所有参数的梯度拼成一个超长向量，然后计算这个向量的模长。
def grad_clipping(net, theta):  #@save
    if isinstance(net, nn.Module):
        params = [p for p in net.parameters() if p.requires_grad]# 列出所有需要计算梯度的参数
    else:
        params = net.params # 列出所有需要计算梯度的参数

    # 计算梯度的 L2 范数（整体梯度的“长度”）
    # 这相当于把所有参数的梯度拼成一个超长向量，然后计算这个向量的模长。
    norm = torch.sqrt(sum(torch.sum((p.grad ** 2)) for p in params))
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm
    # [:] 表示对张量内所有元素进行索引。
    # *= 是原地操作符（in-place），它不会创建新张量，而是直接修改原始张量内部的数据。
    
    # 缩放因子是 theta / norm，这样缩放后，新的梯度范数正好等于 theta。
    # 使用 param.grad[:] *= ... 是为了原地修改梯度，不影响梯度张量的内存地址（这对优化器很重要）。
    # 假设：所有参数梯度拼起来的 L2 范数是 norm = 10.0，设定阈值 theta = 5.0
    # 那么缩放因子 = 5.0 / 10.0 = 0.5，所有梯度乘以 0.5，最终范数变成 5.0。
    # 这是全局裁剪：所有参数共享同一个缩放因子，保持梯度方向不变，只缩放大小。
    # PyTorch 官方也提供了类似功能：torch.nn.utils.clip_grad_norm_，功能基本一致。

## 训练

在训练模型之前，让我们[**定义一个函数在一个迭代周期内训练模型**]。
它与我们训练 :numref:`sec_softmax_scratch`模型的方式有三个不同之处。

1. 序列数据的不同采样方法（随机采样和顺序分区）将导致隐状态初始化的差异。
1. 我们在更新模型参数之前裁剪梯度。
   这样的操作的目的是，即使训练过程中某个点上发生了梯度爆炸，也能保证模型不会发散。
1. 我们用困惑度来评价模型。如 :numref:`subsec_perplexity`所述，
   这样的度量确保了不同长度的序列具有可比性。

具体来说，当使用顺序分区时，
我们只在每个迭代周期的开始位置初始化隐状态。
由于下一个小批量数据中的第$i$个子序列样本
与当前第$i$个子序列样本相邻，
因此当前小批量数据最后一个样本的隐状态，
将用于初始化下一个小批量数据第一个样本的隐状态。
这样，存储在隐状态中的序列的历史信息
可以在一个迭代周期内流经相邻的子序列。
然而，在任何一点隐状态的计算，
都依赖于同一迭代周期中前面所有的小批量数据，
这使得梯度计算变得复杂。
为了降低计算量，在处理任何一个小批量数据之前，
我们先分离梯度，使得隐状态的梯度计算总是限制在一个小批量数据的时间步内。

当使用随机抽样时，因为每个样本都是在一个随机位置抽样的，
因此需要为每个迭代周期重新初始化隐状态。
与 :numref:`sec_softmax_scratch`中的
`train_epoch_ch3`函数相同，
`updater`是更新模型参数的常用函数。
它既可以是从头开始实现的`d2l.sgd`函数，
也可以是深度学习框架中内置的优化函数。


In [ ]:
# 训练网络一个迭代周期，包含多个 batch,每个 batch 计算一次平均损失
# 然后根据平均损失更新一次模型参数 w 和 b
def train_epoch_ch8(net, train_iter, loss, updater, device, use_random_iter):
    """训练网络一个迭代周期（定义见第8章）"""
    state, timer = None, d2l.Timer()
    metric = d2l.Accumulator(2)  # 训练损失之和,词元数量
    for X, Y in train_iter:
        # PyTorch 内置的 GRU 模块 ：其隐藏状态 state 是一个单独的张量（形状为 [层数*方向数, 批量大小, 隐藏单元数]）， 不是元组
        # PyTorch 内置的 LSTM 模块 ：其状态是一个包含两个张量的元组 (h, c)，分别表示隐藏状态和细胞状态
        # 自定义 RNN 实现 ：通常也将状态设计为元组形式，以保持接口一致性
        if state is None or use_random_iter:
            # 在第一次迭代或使用随机抽样时初始化state
            state = net.begin_state(batch_size=X.shape[0], device=device)
        else:
            if isinstance(net, nn.Module) and not isinstance(state, tuple):
                # state对于nn.GRU是个张量
                state.detach_()
            else:
                # 对于LSTM（状态是元组包含h和c）或自定义模型
                # 遍历状态中的每个张量并执行梯度分离
                for s in state:
                    s.detach_()

        # 在转置前，Y 的形状推测为 (批量大小, 时间步数)
        y = Y.T.reshape(-1)  # 转置后，形状为 (时间步数, 批量大小)，然后展平为 (时间步数 * 批量大小,)
        X, y = X.to(device), y.to(device)
        y_hat, state = net(X, state)        # 1.前向传播：不改变 w 和 b
        # 在没有计算 mean之前，loss 的返回值 shape 是 (时间步数 * 批量大小,)
        # 计算 mean 之后，loss 的返回值 shape 是 (1,)，即l是一个标量
        l = loss(y_hat, y.long()).mean()    # 2.计算损失：计算预测值 y_hat 与真实标签 y 之间的损失
        if isinstance(updater, torch.optim.Optimizer):
            updater.zero_grad()             # 3.梯度清零：将 w 和 b 的梯度设为 0
            l.backward()                    # 4.反向传播：计算 w 和 b 的梯度(变化方向)，不改变 w 和 b
            grad_clipping(net, 1)           # 5.梯度裁剪：将 w 和 b 的梯度裁剪到 [-1, 1] 之间
            updater.step()                  # 6.更新参数: 根据梯度更新 w 和 b
        else:
            l.backward()
            grad_clipping(net, 1)
            # 因为已经调用了mean函数
            # 我们已经对损失进行了平均化(调用了.mean())，损失值已经是一个标量
            # 因此，此时传入 batch_size=1 是正确的选择，避免对已经平均化的损失进行二次缩放
            updater(batch_size=1)
        metric.add(l * y.numel(), y.numel())
    return math.exp(metric[0] / metric[1]), metric[1] / timer.stop()
    # 返回困惑度、速度（词元数量/秒）

[**循环神经网络模型的训练函数既支持从零开始实现，
也可以使用高级API来实现。**]


In [ ]:
# 边训练 + 边预测
def train_ch8(net, train_iter, vocab, lr, num_epochs, device, use_random_iter=False):
    """训练模型（定义见第8章）"""
    loss = nn.CrossEntropyLoss()
    animator = d2l.Animator(xlabel='epoch', ylabel='perplexity',
                            legend=['train'], xlim=[10, num_epochs])
    # 定义优化器
    if isinstance(net, nn.Module):
        updater = torch.optim.SGD(net.parameters(), lr)
    else:
        updater = lambda batch_size: d2l.sgd(net.params, lr, batch_size)
    
    # 定义预测函数，使用 predict_ch8 函数对给定前缀进行预测，生成长度为 50 的文本
    predict = lambda prefix: predict_ch8(prefix, 50, net, vocab, device)
    # 训练 + 预测
    for epoch in range(num_epochs):
        # ppl: 困惑度, speed: 速度（词元数量/秒）
        ppl, speed = train_epoch_ch8(net, train_iter, loss, updater, device, use_random_iter)
        if (epoch + 1) % 10 == 0: # 每训练 10 轮，打印一次预测结果
            print(predict('time traveller'))
            animator.add(epoch + 1, [ppl])
        print(f'困惑度 {ppl:.1f}, {speed:.1f} 词元/秒 {str(device)}')

    print(predict('time traveller'))
    print(predict('traveller'))
    print(predict('hello'))

[**现在，我们训练循环神经网络模型。**]
因为我们在数据集中只使用了10000个词元，
所以模型需要更多的迭代周期来更好地收敛。


In [ ]:
# 初始化net
net = RNNModelScratch(len(vocab), num_hiddens, d2l.try_gpu(), get_params, init_rnn_state, rnn)
num_epochs, lr = 100, 1
# 使用顺序迭代训练模型
train_ch8(net, train_iter, vocab, lr, num_epochs, d2l.try_gpu())

[**最后，让我们检查一下使用随机抽样方法的结果。**]


In [ ]:
# 初始化net
net = RNNModelScratch(len(vocab), num_hiddens, d2l.try_gpu(), get_params, init_rnn_state, rnn)
# 使用随机迭代训练模型
num_epochs, lr = 100, 1
train_ch8(net, train_iter, vocab, lr, num_epochs, d2l.try_gpu(), use_random_iter=True)


从零开始实现上述循环神经网络模型，
虽然有指导意义，但是并不方便。
在下一节中，我们将学习如何改进循环神经网络模型。
例如，如何使其实现地更容易，且运行速度更快。

## 小结

* 我们可以训练一个基于循环神经网络的字符级语言模型，根据用户提供的文本的前缀生成后续文本。
* 一个简单的循环神经网络语言模型包括输入编码、循环神经网络模型和输出生成。
* 循环神经网络模型在训练以前需要初始化状态，不过随机抽样和顺序划分使用初始化方法不同。
* 当使用顺序划分时，我们需要分离梯度以减少计算量。
* 在进行任何预测之前，模型通过预热期进行自我更新（例如，获得比初始值更好的隐状态）。
* 梯度裁剪可以防止梯度爆炸，但不能应对梯度消失。

## 练习

1. 尝试说明独热编码等价于为每个对象选择不同的嵌入表示。
1. 通过调整超参数（如迭代周期数、隐藏单元数、小批量数据的时间步数、学习率等）来改善困惑度。
    * 困惑度可以降到多少？
    * 用可学习的嵌入表示替换独热编码，是否会带来更好的表现？
    * 如果用H.G.Wells的其他书作为数据集时效果如何，
      例如[*世界大战*](http://www.gutenberg.org/ebooks/36)？
1. 修改预测函数，例如使用采样，而不是选择最有可能的下一个字符。
    * 会发生什么？
    * 调整模型使之偏向更可能的输出，例如，当$\alpha > 1$，从$q(x_t \mid x_{t-1}, \ldots, x_1) \propto P(x_t \mid x_{t-1}, \ldots, x_1)^\alpha$中采样。
1. 在不裁剪梯度的情况下运行本节中的代码会发生什么？
1. 更改顺序划分，使其不会从计算图中分离隐状态。运行时间会有变化吗？困惑度呢？
1. 用ReLU替换本节中使用的激活函数，并重复本节中的实验。我们还需要梯度裁剪吗？为什么？


[Discussions](https://discuss.d2l.ai/t/2103)
